In [30]:
import os
import pandas as pd
import preprocessing.pipe as pipe
import regex as re
import numpy as np

In [31]:
#directories boredom
os.chdir("D:/Drive/FHNW/zaRepos/fhnw_ds_fs2021_medical_challenge/preprocessing")
rel_path = "../data/uveitis_data.xlsx"

#variables assignement
list_path = "../data/col_names&data_type-Copy1.xlsx"
col_index_name = "new col name"
col_data_type_name = "data_type"
data_type = "numerical"

desired_dtype = ["Int64", "float64"]



In [32]:
#read data
df1 = pipe.read_data(rel_path)

#rename columns of df1 according to external excel source
df1 = pipe.rename(df1, list_path)
print(list(df1.columns))

#return list of all columns with specific dtype
num_columns = pipe.list_of_totype(list_path, col_index_name, col_data_type_name, data_type)


df1

uveitis_data.xlsx will be read
reading filetype .xlsx

Successfully created table with  126850 values and loaded as df
The table is 118 wide and 1075 long 

['id', 'gender', 'race', 'loc', 'cat', 'other_', 'ehr_diagnosis', 'specific_diagnosis', 'notes', 'ac_abn_od_cells', 'ac_abn_os_cells', 'vit_abn_od_cells', 'vit_abn_os_cells', 'vit_abn_od_haze', 'vit_abn_os_haze', 'calcium', 'uom', 'range', 'lactate_dehydrogenase', 'uom1', 'range1', 'c-reactive_protein,_normal_and_high_sensitivity', 'uom2', 'range2', 'wbc', 'uom3', 'range3', 'rbc', 'uom4', 'range4', 'hemoglobin', 'uom5', 'range5', 'hematocrit', 'uom6', 'range6', 'mcv', 'uom7', 'range7', 'mch', 'uom8', 'range8', 'mchc', 'uom9', 'range9', 'rdw', 'uom10', 'range10', 'platelet_count', 'uom11', 'range11', 'neutrophil_%', 'uom12', 'range12', 'lymphocytes_%', 'uom13', 'range13', 'angiotensin_conv#enzyme', 'uom14', 'range14', 'beta-2-microglobulin', 'uom15', 'range15', 'lupus_anticoagulant', 'lysozyme,_plasma', 'uom18', 'range18', 'anti-ccp

,id,gender,race,loc,cat,other_,ehr_diagnosis,specific_diagnosis,notes,ac_abn_od_cells,...,range34,proteinase-3_antibodies,uom35,range35,rheumatoid_factor,uom36,range36,hbc__ab,hbs__ag,hcv__ab
0,1,Male,Black/African American,Anterior,Nonneoplastic Masquerade,NaN,Posner-Schlossman ...,Idiopathic_anterior,NaN,0,...,NaN,NaN,NaN,NaN,<15,IU/mL,<15,NEGATIVE,NEGATIVE,NEGATIVE
1,2,Female,Black/African American,Intermediate,Idiopathic,NaN,Ant/Int Uveitis ...,pars planitis,pars planitis,2,...,<0.4 (Negative),<0.2,U,<0.4 (Negative),<10,IU/mL,<13,NEGATIVE,NEGATIVE,NEGATIVE
2,3,Female,White,Pan,Systemic,NaN,VKH,VKH,NaN,0.5,...,NaN,NaN,NaN,NaN,<10,IU/mL,<13,NEGATIVE,NEGATIVE,NEGATIVE
3,4,Female,White,Anterior,Idiopathic,NaN,chronic idiopathic anterior uveitis ...,Idiopathic_anterior,NaN,0,...,NaN,NaN,NaN,NaN,<15,IU/mL,<15,NEGATIVE,NEGATIVE,NEGATIVE
4,5,Female,Unknown Race,Anterior,Idiopathic,NaN,Acute Anterior Uveitis ...,Idiopathic_anterior,NaN,0,...,NaN,NaN,NaN,NaN,<10,IU/mL,<13,NEGATIVE,NEGATIVE,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,1071,Male,Asian,Pan,Infectious,granulomatous,TB assoc granulomatous panuveitis,Tuberculosis,NaN,0,...,<0.4 (Negative),0.7,U,<0.4 (Negative),<15,IU/mL,<15,NEGATIVE,NEGATIVE,NEGATIVE
1071,1072,Male,Unknown Race,Intermediate,Idiopathic,NaN,NG anterior and intermediate uveitis,pars planitis,pars planitis,0,...,NaN,NaN,NaN,NaN,<15,IU/mL,<15,NEGATIVE,NEGATIVE,NEGATIVE
1072,1073,Female,Unknown Race,Posterior,Infectious,NaN,toxoplasmosis retinochoroiditis,Toxoplasmosis,NaN,0,...,NaN,NaN,NaN,NaN,<15,IU/mL,<15,NEGATIVE,NEGATIVE,NEGATIVE
1073,1074,Female,Asian,Posterior,WDS,NaN,ampiginous-like chorioretinopathy,WDS,WDS,0,...,NaN,NaN,NaN,NaN,<15,IU/mL,<15,NEGATIVE,NEGATIVE,NEGATIVE


In [33]:
#function to return list of columns to convert to which data type (with choice) according to given list


def list_of_totype(list_path, col_index_name, col_data_type_name="data_type", data_type="numerical"):
    """
    Return list of columns, selected based on their data types from custom excel list
    -----
    :param list_path: path of excel document to read data type from
    :param col_index_name: index column to use (column name of data type)
    :param col_data_type_name: name of column where data type is stored
    :param data_type: type of data desired. Should be numerical, categorical, char or both (appelation for edge cases)

    :return: returns a list of column names with the desired data type
    """
    col = pd.read_excel(list_path, index_col = col_index_name)[col_data_type_name] #get content of column with name data_type
    col = col.reset_index()

    new_col = col[col_index_name].str.strip().str.lower()
    new_col = [c.replace(' ', '_') for c in new_col] # remove whitespace
    new_col = [re.sub(r"\([^()]*\)", "", c) for c in new_col] # remove all text in parantheses
    col[col_index_name] = new_col
    
    col_to_type = col.where(col.data_type == data_type).dropna()[col_index_name].tolist()
    return col_to_type

item_filter = list_of_totype(list_path, col_index_name, col_data_type_name, data_type)
print(item_filter)

['id', 'calcium', 'lactate_dehydrogenase', 'c-reactive_protein,_normal_and_high_sensitivity', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count', 'neutrophil_%', 'lymphocytes_%', 'angiotensin_conv#enzyme', 'beta-2-microglobulin', 'lysozyme,_plasma', 'anti-dnase_b', 'complement_c3', 'complement_c4', 'rheumatoid_factor']


In [34]:
#this function is pretty useless alone
def problem_columns(matches, desired_dtype):
    """
    Return all columns and their content that couldn't be correctly coerced to a desired dtype
    -----
    :param matches: feeded from another function
    :param desired_dtype: a list or string that contains the desired data_types that were transformed correctly

    :return: returns a dataframe with all the problematic columns
    """
    for key, value in matches.iteritems():
        #print(key, value)
        if value.dtype in desired_dtype:
            print("\n", key, "is desired", value.dtype, "and will be popped from problematic list \n")
            matches.pop(key)
        else:
            print(key, "has unwanted dtype, keeping for transformation")
    return matches

In [35]:
#function to coerce columns to desired datatype
def coerce_then_problems(dframe, list_path, col_index_name, col_data_type_name, data_type, desired_dtype):
    """
    Coerce with convert_dtypes pandas function all columns from the list_of_totype. Those that fail to
    be converted into the desired dtype will be compiled into a dataframe for next steps
    -----
    :param dframe: dataframe to be checked and wrecked
    :param list_path: path of excel document to read data type from
    :param col_index_name: index column to use (column name of data type)
    :param col_data_type_name: name of column where data type is stored
    :param data_type: type of data desired. Should be numerical, categorical, char or both (appelation for edge cases)
    :param desired_dtype: a list or string that contains the desired data_types that were transformed correctly

    :return: a dataframe with values that couldn't be coerced to the desired data_type automagically via convert_dtypes()
    """

    item_filter = list_of_totype(list_path, col_index_name, col_data_type_name, data_type)
    matches = dframe.filter(items = item_filter).convert_dtypes()
    print(matches)
    matches = problem_columns(matches, desired_dtype)
    return matches

In [36]:
#create dataframe with columns that contain a mix of strings and numerical values
problem_df = coerce_then_problems(df1, list_path, col_index_name, col_data_type_name, data_type, desired_dtype)
problem_columns = list(problem_df)
print(problem_columns)

#function that iterates trough columns given by previous function to coerce values that aren't numerical in nature to something else (0, , NaN, NA, 99)


        id calcium lactate_dehydrogenase  \
0        1    2.27                   184   
1        2    2.32                   209   
2        3    2.44                   194   
3        4    2.45                   156   
4        5    2.28                   175   
...    ...     ...                   ...   
1070  1071    2.26                   194   
1071  1072    2.36                   239   
1072  1073    2.26                   196   
1073  1074    2.41                   173   
1074  1075    2.39                   214   

     c-reactive_protein,_normal_and_high_sensitivity    wbc   rbc hemoglobin  \
0                                               1.83   6.91  4.97       14.6   
1                                              <0.15   4.24  4.22         11   
2                                              0.137   6.38  4.31       13.6   
3                                                3.5   5.24  4.82       14.6   
4                                                  1   8.31  4.72      

C:\Users\Lupo_Remastered\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  del sys.path[0]
